<a href="https://colab.research.google.com/github/choiboy92/ImperialML/blob/main/Coursework/ML_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

url = 'https://raw.githubusercontent.com/choiboy92/ImperialML/main/Coursework/dataset1.csv?token=AGM5IPSUP6D3CRL3524FTZ3BSZ4ZI'
df = pandas.read_csv(url)
arm_length = np.array(df['Arm length (m)'])
weight = np.array(df['Ball weight (kg)'])
radius = np.array(df['Ball radius (mm)'])
temp = np.array(df['Air temperature (deg C)'])
k = np.array(df['Spring constant (N per m)'])
dev_weight = np.array(df['Device weight (kg)'])
y = np.array(df['Target hit'])
print(y)

[0. 1. 1. ... 1. 1. 1.]


In [17]:
scArray = np.array([[np.mean(arm_length), np.mean(weight), np.mean(radius), np.mean(temp), np.mean(k), np.mean(dev_weight)],
                   [np.std(arm_length), np.std(weight), np.std(radius), np.std(temp), np.std(k), np.std(dev_weight)]])

np.savetxt('scaleParams.txt', scArray)

In [18]:
# download it from the Colab interface
from google.colab import files
files.download('scaleParams.txt')

loadedScales = np.loadtxt('scaleParams.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
# Normalise input datasets

X_unscaled = np.concatenate((arm_length.reshape(2000,1), weight.reshape(2000,1), radius.reshape(2000,1), temp.reshape(2000,1), k.reshape(2000,1), dev_weight.reshape(2000,1)), axis=1)
X_scaled = (X_unscaled-loadedScales[0,:])/loadedScales[1,:]

In [45]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

#set up a sequential neural network
model = Sequential()

model.add(Dense(units=4, activation='relu', input_dim=6)) 
model.add(Dense(units=4, activation='relu')) 
model.add(Dense(units=2, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='sgd')

from tensorflow.keras.utils import to_categorical
y_binary = to_categorical(y)
model.fit(X_scaled, y_binary, epochs=250, batch_size=32)

y_val = model.predict(X_scaled)
y_val = np.argmax(y_val, axis=-1)

nTot = len(y_val) 
nMatch = 0 
for i in range(len(y_val)):
  if y[i] == y_val[i]:
    nMatch += 1
print("With Validation data -- Accuracy: ", 100 * nMatch / nTot, "%")


Epoch 1/250
63/63 [==============================] - 1s 1ms/step - loss: 0.7079
Epoch 2/250
63/63 [==============================] - 0s 1ms/step - loss: 0.6327
Epoch 3/250
63/63 [==============================] - 0s 1ms/step - loss: 0.5923
Epoch 4/250
63/63 [==============================] - 0s 1ms/step - loss: 0.5583
Epoch 5/250
63/63 [==============================] - 0s 1ms/step - loss: 0.5253
Epoch 6/250
63/63 [==============================] - 0s 1ms/step - loss: 0.4915
Epoch 7/250
63/63 [==============================] - 0s 1ms/step - loss: 0.4576
Epoch 8/250
63/63 [==============================] - 0s 2ms/step - loss: 0.4252
Epoch 9/250
63/63 [==============================] - 0s 1ms/step - loss: 0.3946
Epoch 10/250
63/63 [==============================] - 0s 2ms/step - loss: 0.3666
Epoch 11/250
63/63 [==============================] - 0s 2ms/step - loss: 0.3419
Epoch 12/250
63/63 [==============================] - 0s 1ms/step - loss: 0.3211
Epoch 13/250
63/63 [=================